In [2]:
import csv
import time
import random
import re
from typing import Union

with open('laptops.csv') as f:
    reader = csv.reader(f)
    rows = list(reader)
    header = rows[0]
    rows = rows[1:]

def row_price(row) -> float:
    """
    Retorna o preço de uma linha.

    Args:
        row (list): Uma lista representando uma linha de dados.

    Returns:
        float: O preço da linha.
    """
    return row[-1]


In [ ]:
class Inventory:
    def __init__(self, csv_filename):
        """
        Inicializa a classe Inventory com os dados do arquivo CSV.

        Args:
            csv_filename (str): O nome do arquivo CSV contendo os dados.

        """
        with open('laptops.csv') as f:
            reader = csv.reader(f)
            rows = list(reader)
        self.header = rows[0]
        self.rows = rows[1:]
        
        for row in self.rows:
            row[-1] = int(row[-1])
        
        self.id_to_row = dict()
        for row in self.rows:
            self.id_to_row[row[0]] = row
        
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])

        self.rows_by_price = sorted(self.rows, key=row_price)

    def get_laptop_from_id(self, laptop_id) -> Union[list, None]:
        """
        Obtém os detalhes de um laptop pelo ID.

        Args:
            laptop_id (str): O ID do laptop a ser encontrado.

        Returns:
            list: Uma lista contendo os detalhes do laptop encontrado, ou None se não for encontrado.
        """
        for row in self.rows:
            if row[0] == laptop_id:
                return row
        return None
    
    def get_laptop_if_fast(self, laptop_id) -> Union[list, None]:
        """
        Obtém os detalhes de um laptop pelo ID de forma mais eficiente.

        Args:
            laptop_id (str): O ID do laptop a ser encontrado.

        Returns:
            list: Uma lista contendo os detalhes do laptop encontrado, ou None se não for encontrado.
        """
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars) -> bool:
        """
        Verifica se um preço específico está disponível ou se pode ser alcançado somando dois preços diferentes.

        Args:
            dollars (float): O preço a ser verificado.

        Returns:
            bool: True se o preço ou a combinação de preços for encontrada, False caso contrário.
        """
        for item in self.rows:
            if item[-1] == dollars:
                return True
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1]+row2[-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars) -> bool:
        """
        Verifica se um preço específico está disponível de forma mais eficiente.

        Args:
            dollars (float): O preço a ser verificado.

        Returns:
            bool: True se o preço for encontrado, False caso contrário.
        """
        if dollars in self.prices:
            return True
        for price in self.prices:
            if dollars - price in self.prices:
                return True
        return False
    
    def find_laptop_with_price(self, target_price) -> int:
        """
        Encontra um laptop com um preço específico de forma eficiente usando busca binária.

        Args:
            target_price (float): O preço do laptop a ser encontrado.

        Returns:
            int: O índice do laptop encontrado ou -1 se não for encontrado.
        """
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            value = self.rows_by_price[range_middle][-1]
            if value == target_price:                            
                return range_middle                        
            elif value < target_price:                           
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle - 1 
        if self.rows_by_price[range_start][-1] != target_price:                  
            return -1                                      
        return range_start
    
    def find_first_laptop_more_expensive(self, target_price) -> int:
        """
        Encontra o primeiro laptop com preço superior a um valor específico de forma eficiente.

        Args:
            target_price (float): O preço mínimo a ser considerado.

        Returns:
            int: O índice do primeiro laptop encontrado ou -1 se não for encontrado.
        """
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start
        
    def find_laptops_in_price_range(self, min_price, max_price) -> list:
        """
        Encontra laptops dentro de uma faixa de preço específica.

        Args:
            min_price (float): O preço mínimo da faixa.
            max_price (float): O preço máximo da faixa.

        Returns:
            list: Uma lista de laptops que estão dentro da faixa de preço.
        """
        laptops_affordable = []

        for row in self.rows_by_price:
            if min_price <= row[-1] <= max_price:
                laptops_affordable.append(row)

        return laptops_affordable      

    
    def find_laptops_in_price_range_fast(self, min_price, max_price) -> list:
        """
        Encontra laptops dentro de uma faixa de preço específica de forma eficiente.

        Args:
            min_price (float): O preço mínimo da faixa.
            max_price (float): O preço máximo da faixa.

        Returns:
            list: Uma lista de laptops que estão dentro da faixa de preço.
        """
        laptops_in_range = []  # Inicializa uma lista vazia para armazenar laptops dentro da faixa de preço

        # Encontra o índice do primeiro laptop com preço maior ou igual a min_price
        start_index = self.find_first_laptop_more_expensive(min_price)

        # Se não houver laptops nessa faixa de preço, retorna a lista vazia
        if start_index == -1:
            return laptops_in_range

        # Percorre a lista ordenada de laptops até encontrar laptops fora da faixa de preço
        while start_index < len(self.rows_by_price):
            price = self.rows_by_price[start_index][-1]

            # Se o preço do laptop for maior do que o preço máximo, interrompe o loop
            if price > max_price:
                break

            # Adiciona o laptop à lista de laptops dentro da faixa de preço
            laptops_in_range.append(self.rows_by_price[start_index])
            start_index += 1

        # Retorna a lista de laptops dentro da faixa de preço
        return laptops_in_range

    
    def is_laptop_match(self, laptop, ram, storage) -> bool:
        """
        Verifica se um laptop atende a especificações de RAM e armazenamento.

        Args:
            laptop (list): Uma lista representando os detalhes do laptop.
            ram (int): A quantidade mínima de RAM desejada.
            storage (int): A quantidade mínima de armazenamento desejada.

        Returns:
            bool: True se o laptop atender às especificações, False caso contrário.
        """
        # Encontra correspondências para a quantidade de RAM na descrição do laptop (laptop[7])
        matches_ram = re.findall(r'\d+', laptop[7])

        # Se houver correspondências, converte a primeira correspondência em um número inteiro
        if matches_ram:
            laptop_ram = int(matches_ram[0])
        else:
            # Se não houver correspondências, imprime uma mensagem de erro e retorna 0 (RAM inválida)
            print('Invalid RAM format')
            return False

        # Encontra correspondências para a capacidade de armazenamento na descrição do laptop (laptop[8])
        matches_storage = re.findall(r'\d+', laptop[8])

        # Se houver correspondências, converte a primeira correspondência em um número inteiro
        if matches_storage:
            laptop_storage = int(matches_storage[0])
        else:
            # Se não houver correspondências, imprime uma mensagem de erro e retorna 0 (capacidade de armazenamento inválida)
            print('Invalid Memory format')
            return False

        # Verifica se o laptop atende às especificações de RAM e armazenamento
        return laptop_ram >= ram and laptop_storage >= storage

    def find_cheapest_laptop_with_specifications(self, ram, storage) -> Union[list, None]:
        """
        Encontra o laptop mais barato que atende às especificações de RAM e armazenamento.

        Args:
            ram (int): A quantidade mínima de RAM desejada.
            storage (int): A quantidade mínima de armazenamento desejada.

        Returns:
            list: Uma lista representando os detalhes do laptop mais barato que atende às especificações,
            ou None se nenhum laptop for encontrado.
            None
        """
        # Filtra os laptops que atendem às especificações usando a função is_laptop_match
        matching_laptops = [laptop for laptop in self.rows if self.is_laptop_match(laptop, ram, storage)]

        # Se não houver laptops que atendam às especificações, retorna None
        if not matching_laptops:
            return None

        # Encontra o laptop mais barato entre os laptops que atendem às especificações
        cheapest_laptop = min(matching_laptops, key=lambda laptop: laptop[-1])

        # Retorna os detalhes do laptop mais barato encontrado
        return cheapest_laptop


## Testing

### Uploading data

In [3]:
inventory_instance = Inventory('laptop.csv')
print(inventory_instance.header)
print(len(inventory_instance.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303


### Get Laptop ID

In [6]:
inventory_instance = Inventory('laptop.csv')
print(inventory_instance.header)
print(len(inventory_instance.rows))
print(inventory_instance.get_laptop_from_id('3362737'))
print(inventory_instance.get_laptop_from_id('3362736'))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
1303
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


### Improving Performance

In [60]:
inventory_instance = Inventory('laptop.csv')
print(inventory_instance.header[8])
print(len(inventory_instance.rows))
print(inventory_instance.get_laptop_if_fast('3362737'))
print(inventory_instance.get_laptop_if_fast('3362736'))

Memory
1303
['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]
None


### Comparing Times

In [11]:
ids = [str(random.randint(1000000, 9999999)) for _ in range(10000)] # step 3

inventory_instance = Inventory('laptops.csv')                                # step 4

total_time_no_dict = 0
total_time_dict = 0

for item in ids:
    start = time.time()
    inventory_instance.get_laptop_from_id(item)
    end = time.time()
    total_time_no_dict = end-start

for item in ids:
    start = time.time()
    inventory_instance.get_laptop_if_fast(item)
    end = time.time()
    total_time_dict = end-start
    
print('Sem dict: ', total_time_no_dict, '\nCom dict: ', total_time_dict)

Sem dict:  0.00015687942504882812 
Com dict:  7.152557373046875e-07


In [13]:
inventory_instance = Inventory('laptop.csv')
print(inventory_instance.check_promotion_dollars(1000))
print(inventory_instance.check_promotion_dollars(442))

True
False


### Optimizing

In [16]:
inventory_instance = Inventory('laptop.csv')
print(inventory_instance.check_promotion_dollars_fast(1000))
print(inventory_instance.check_promotion_dollars_fast(442))

True
False


### Comparing Times

In [18]:
prices = [random.randint(100, 5000) for _ in range(100)] # step 3

inventory_instance = Inventory('laptops.csv')                                # step 4

total_time_no_set = 0
total_time_set = 0

for item in prices:
    start = time.time()
    inventory_instance.check_promotion_dollars(item)
    end = time.time()
    total_time_no_set = end-start

for item in prices:
    start = time.time()
    inventory_instance.check_promotion_dollars_fast(item)
    end = time.time()
    total_time_set = end-start
    
print('Sem set: ', total_time_no_set, '\nCom set: ', total_time_set)

Sem set:  0.0004892349243164062 
Com set:  9.5367431640625e-07


### Finding Budget

In [21]:
inventory_instance = Inventory('laptop.csv')
print(inventory_instance.find_first_laptop_more_expensive(1000))
print(inventory_instance.find_first_laptop_more_expensive(10000))

683
-1


In [108]:
inventory = Inventory('laptops.csv')
cheapest_laptop = inventory.find_cheapest_laptop_with_specifications(8, 256)
if cheapest_laptop:
    print("O laptop mais barato com 8GB de RAM e 256GB de armazenamento é:", cheapest_laptop)
else:
    print("Nenhum laptop corresponde às especificações.")


O laptop mais barato com 8GB de RAM e 256GB de armazenamento é: ['2618101', 'Acer', 'ES1-523-84K7 (A8-7410/8GB/256GB/FHD/W10)', 'Notebook', '15.6', 'Full HD 1920x1080', 'AMD A8-Series 7410 2.2GHz', '8GB', '256GB SSD', 'AMD Radeon R5', 'Windows 10', '2.23kg', 469]


In [58]:
import numpy as np

min_prices = [random.randint(100, 5000) for _ in range(100)] # step 3
max_prices = [random.randint(5000, 10000) for _ in range(100)] # step 3

inventory_instance = Inventory('laptops.csv')                                # step 4

total_time_no_binary_lst = []
total_time_no_binary = 0
total_time_binary_lst = []
total_time_binary = 0

for idx, item_min in enumerate(min_prices):
    start = time.time()
    inventory_instance.find_laptops_in_price_range(item_min, max_prices[idx])
    end = time.time()
    total_time_no_binary = end - start
    total_time_no_binary_lst.append(total_time_no_binary)

for idx, item_min in enumerate(min_prices):
    start = time.time()
    inventory_instance.find_laptops_in_price_range_fast(item_min, max_prices[idx])
    end = time.time()
    total_time_binary = end - start
    total_time_binary_lst.append(total_time_binary)

    
print('Sem busca binária: ', total_time_no_binary, '\nCom busca binária: ', total_time_binary)
print('\nTempo médio sem busca binária: ', min(total_time_no_binary_lst), '\nCom busca binária: ', min(total_time_binary_lst))

Sem busca binária:  0.0003285408020019531 
Com busca binária:  0.0007865428924560547

Tempo médio sem busca binária:  0.00013756752014160156 
Com busca binária:  8.58306884765625e-06


### Extra

In [22]:
inventory_instance = Inventory('laptop.csv')
print(inventory_instance.find_laptops_in_price_range_fast(1000,10000))
#print(inventory_instance.find_first_laptop_more_expensive(10000))

[['8747948', 'Lenovo', 'ThinkPad T460', 'Notebook', '14', '1366x768', 'Intel Core i5 6200U 2.3GHz', '4GB', '508GB Hybrid', 'Intel HD Graphics 520', 'Windows 7', '1.70kg', 1002], ['5550925', 'Dell', 'Latitude 5580', 'Notebook', '15.6', '1366x768', 'Intel Core i5 7300U 2.6GHz', '8GB', '500GB HDD', 'Intel HD Graphics 620', 'Windows 10', '1.9kg', 1008], ['3667708', 'Acer', 'Aspire F5-573G-510L', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '12GB', '128GB SSD +  1TB HDD', 'Nvidia GeForce GTX 950M', 'Windows 10', '2.4kg', 1009], ['8017281', 'Dell', 'Vostro 5568', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i7 7500U 2.7GHz', '8GB', '256GB SSD', 'Nvidia GeForce 940MX', 'Windows 10', '2.18kg', 1009], ['6766298', 'Lenovo', 'Thinkpad 13', 'Notebook', '13.3', 'IPS Panel Full HD 1920x1080', 'Intel Core i7 7500U 2.7GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'Windows 10', '1.4kg', 1010], ['9303831', 'HP', 'ProBook 440', 'Notebook', '14', 'Full HD 1920x1080',